In [1]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

import argparse
import matplotlib.pyplot as plt
import numpy as np
import pylab
import pickle
import random
import types

from qiskit import Aer, BasicAer
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.algorithms import ExactEigensolver, VQE
from qiskit.aqua.components.optimizers import SPSA, COBYLA, L_BFGS_B
from qiskit.aqua.components.variational_forms import RY, RYRZ
#from qiskit.aqua.operators import Z2Symmetries
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry.core import Hamiltonian, TransformationType, QubitMappingType
from time import time

args = types.SimpleNamespace()
#args.algorithm='VQE' 
#args.basis_set='321++g'
#args.basis_set='431g'
#args.basis_set='321g'
#args.basis_set='ano'
#args.basis_set='augpc0'
#args.basis_set='ccpvdz' 
#args.basis_set='def2svp'
#args.basis_set='demon'
#args.basis_set='dz'
#args.basis_set='dzp'
#args.basis_set='minao'
#args.basis_set='pc0'
#args.basis_set='pcseg0'
#args.basis_set='sto6g'
#args.basis_set='stuttgart'
#args.basis_set='tzv'

args.max_parallel_threads=10 
args.molecule='H2' 
args.num_shots=1000
args.outpath_mol='/home/ubuntu/mol2qpu/output/operators/molecule/'
args.outpath_ferm='/home/ubuntu/mol2qpu/output/operators/fermionic/'
args.outpath_qub='/home/ubuntu/mol2qpu/output/operators/qubit/'
#args.outpath_vqe='/home/ubuntu/mol2qpu/output/VQE_results/'
args.outpath_vqe_H2 = '/home/ubuntu/mol2qpu/output/VQE_results/H2/'
args.outpath_vqe_H2_opts = '/home/ubuntu/mol2qpu/output/VQE_results/H2/'

args.qubitmapping_type='bravyi_kitaev'
#args.qubitmapping_type='jordan_wigner'
#args.qubitmapping_type='parity'
#args.random_seed=750 
#args.two_qubit_reduce = True
#args.vqe_aer = True
args.vqe_depth=1 
args.vqe_entangler='linear' 
#args.vqe_entangler='full' 
#args.vqe_max_iter=2
#args.vqe_opt_params = False
#args.vqe_optimizer='SPSA' 
#args.vqe_sim = True
#args.vqe_var_form='RY' 

## load qmolecule
d_string = '0pt5'
#filename_mol = args.outpath_mol + args.molecule + '_' + args.basis_set + '_' + d_string + '_MOLE' + '.pkl'
#qmolecule = pickle.load(open(filename_mol,'rb'))

## load operators
#filename_ferm = args.outpath_ferm + args.molecule + '_' + args.basis_set + '_' + d_string + '_FERM' + '.pkl'
#filename_qub = args.outpath_qub + args.molecule + '_' + args.basis_set + '_' + d_string + '_QUBIT_bk' + '.pkl'
#ferOp = pickle.load(open(filename_ferm, 'rb'))
#qubitOp = pickle.load(open(filename_qub, 'rb'))
molecule = 'H .0 .0 -0.25; H .0 .0 0.25'
basis_sets = ['sto3g', '321g']
optimizers = [COBYLA, L_BFGS_B, SPSA]

In [2]:
converge_cnts = np.empty(shape=[len(basis_sets), len(optimizers)], dtype=object)
converge_vals = np.empty(shape=[len(basis_sets), len(optimizers)], dtype=object)
eigvecs = np.empty(shape=[len(basis_sets), len(optimizers)], dtype=object)

#print(range(len(basis_sets)))
#print(range(len(optimizers)))

for i in range(len(basis_sets)):
    print("i ======= ", i)

    driver = PySCFDriver(molecule, basis=basis_sets[i])
    qmolecule = driver.run()
    operator =  Hamiltonian(transformation=TransformationType.FULL,
                            qubit_mapping=QubitMappingType.PARITY,
                            two_qubit_reduction=False,
                            freeze_core=False,
                            orbital_reduction=None)
    qubit_op, aux_ops = operator.run(qmolecule)
    num_particles = qmolecule.num_alpha + qmolecule.num_beta
    
    print("H2, basis_set = ", basis_sets[i])
    print("num orbitals = ", qmolecule.num_orbitals)
    print("num_particles = ", num_particles)
    print("num qubits = ", qubit_op.num_qubits)
        
    for j in range(len(optimizers)):
        print("j ======= ", j)
        aqua_globals.random_seed = 250
        optimizer = optimizers[j]()
        print('\rOptimizer: {}        '.format(type(optimizer).__name__), end='\n')
        
#         #var_form = RY(num_qubits, initial_state=init_state)
        var_form = RYRZ(qubit_op.num_qubits, depth=args.vqe_depth, entanglement=args.vqe_entangler) 

        counts = []
        values = []
        def store_intermediate_result(eval_count, parameters, mean, std):
            counts.append(eval_count)
            values.append(mean)
    
        algo = VQE(qubit_op, var_form, optimizer, callback=store_intermediate_result)
        backend = Aer.get_backend('qasm_simulator')
        quantum_instance = QuantumInstance(circuit_caching=True, backend=backend,
                                           backend_options={'max_parallel_threads': args.max_parallel_threads,                                                                             'max_parallel_experiments': 0, 
                                                            'shots': args.num_shots})
        algo_result = algo.run(quantum_instance)
        converge_cnts[i][j] = np.asarray(counts)
        converge_vals[i][j] = np.asarray(values)
        eigvecs[i][j] = algo_result["min_vector"]
        
        ## save VQE results
        filename_vqe = args.outpath_vqe \
                     + args.molecule + '_' \
                     + basis_sets[i] + '_' \
                     + d_string + '_' \
                     + type(optimizer).__name__  + '_' \
                     + 'RY_lin_bk_' \
                     + str(args.num_shots) + '.pkl'
        filehandler_vqe = open(filename_vqe, 'wb')
        pickle.dump(algo_result, filehandler_vqe)
        print("saved file = ", filename_vqe)
    
    print('\rOptimization complete      ')

i =======  0
H2, basis_set =  sto3g
num orbitals =  2
num_particles =  2
num qubits =  4
j =======  0
Optimizer: COBYLA        
def _run


/home/ubuntu/anaconda3/envs/mol2qpu/lib/python3.8/site-packages/qiskit/aqua/quantum_instance.py:205: UserWarning: circuit_caching will be removed at Qiskit Aqua 0.7+, this setting will be ignored. On the other hand, Qiskit Aqua does support parameterized circuits for adaptive algorithms (e.g. VQE and VQC) to avoid for compiling the circuit with the same topology multiple times
  warnings.warn("circuit_caching will be removed at Qiskit Aqua 0.7+, "


saved file =  /home/ubuntu/mol2qpu/output/VQE_results/H2_sto3g_0pt5_COBYLA_RY_lin_bk_1000.pkl
j =======  1
Optimizer: L_BFGS_B        
def _run
saved file =  /home/ubuntu/mol2qpu/output/VQE_results/H2_sto3g_0pt5_L_BFGS_B_RY_lin_bk_1000.pkl
j =======  2
Optimizer: SPSA        
def _run
saved file =  /home/ubuntu/mol2qpu/output/VQE_results/H2_sto3g_0pt5_SPSA_RY_lin_bk_1000.pkl
Optimization complete      
i =======  1
H2, basis_set =  321g
num orbitals =  4
num_particles =  2
num qubits =  8
j =======  0
Optimizer: COBYLA        
def _run
saved file =  /home/ubuntu/mol2qpu/output/VQE_results/H2_321g_0pt5_COBYLA_RY_lin_bk_1000.pkl
j =======  1
Optimizer: L_BFGS_B        
def _run
saved file =  /home/ubuntu/mol2qpu/output/VQE_results/H2_321g_0pt5_L_BFGS_B_RY_lin_bk_1000.pkl
j =======  2
Optimizer: SPSA        
def _run
saved file =  /home/ubuntu/mol2qpu/output/VQE_results/H2_321g_0pt5_SPSA_RY_lin_bk_1000.pkl
Optimization complete      


In [ ]:
## load VQE results
print(filename_vqe)
#result_load = pickle.load(open(filename_vqe,'rb'))
#result_load

## H2 STO3G basis set

In [ ]:
#args.basis_set='sto3g'
#molecule = 'H .0 .0 -0.25; H .0 .0 0.25'
#driver = PySCFDriver(molecule, basis=args.basis_set)
#qmolecule = driver.run()
#operator =  Hamiltonian(transformation=TransformationType.FULL,
#                        qubit_mapping=QubitMappingType.PARITY,
#                        two_qubit_reduction=False,
#                        freeze_core=False,
#                        orbital_reduction=None)
#qubit_op, aux_ops = operator.run(qmolecule)

In [ ]:
#print("molecule file = ", filename_mol)
#print("qubit file = ", filename_qub)
#num_particles = qmolecule.num_alpha + qmolecule.num_beta
#print("num orbitals = ", qmolecule.num_orbitals)
#print("num_particles = ", num_particles)
#print("num qubits = ", qubit_op.num_qubits)

In [ ]:
#backend = Aer.get_backend('qasm_simulator')
#quantum_instance = QuantumInstance(circuit_caching=True, 
#                                   backend=backend,
#                                   backend_options={'max_parallel_threads': args.max_parallel_threads,                                                                             'max_parallel_experiments': 0, 
#                                                    'shots': args.num_shots})

#optimizer = SPSA(max_trials=50)
#optimizer = COBYLA(maxiter=50)
#var_form = RY(qubit_op.num_qubits, depth=args.vqe_depth, entanglement=args.vqe_entangler) 
#var_form = RYRZ(qubit_op.num_qubits, depth=args.vqe_depth, entanglement=args.vqe_entangler) 


#counts = []
#values = []
#def store_intermediate_result(eval_count, parameters, mean, std):
#    counts.append(eval_count)
#    values.append(mean)
  
#algo = VQE(qubit_op, var_form, optimizer, callback=store_intermediate_result)
#algo = VQE(qubit_op, var_form, optimizer)

optimizers = [COBYLA, L_BFGS_B, SPSA]
#optimizers = [COBYLA]
converge_cnts = np.empty([len(optimizers)], dtype=object)
converge_vals = np.empty([len(optimizers)], dtype=object)
eigvecs = np.empty([len(optimizers)], dtype=object)

for i in range(len(optimizers)):
    aqua_globals.random_seed = 250
    optimizer = optimizers[i]()
    print('\rOptimizer: {}        '.format(type(optimizer).__name__), end='')
    #init_state = Zero(num_qubits)
    #var_form = RY(num_qubits, initial_state=init_state)
    var_form = RYRZ(qubit_op.num_qubits, depth=args.vqe_depth, entanglement=args.vqe_entangler) 

    counts = []
    values = []
    def store_intermediate_result(eval_count, parameters, mean, std):
        counts.append(eval_count)
        values.append(mean)
    
    algo = VQE(qubit_op, var_form, optimizer, callback=store_intermediate_result)
    backend = Aer.get_backend('qasm_simulator')
    quantum_instance = QuantumInstance(circuit_caching=True, backend=backend,
                                       backend_options={'max_parallel_threads': args.max_parallel_threads,                                                                             'max_parallel_experiments': 0, 
                                                        'shots': args.num_shots})

    algo_result = algo.run(quantum_instance)
    converge_cnts[i] = np.asarray(counts)
    converge_vals[i] = np.asarray(values)
    #eigvecs[i] = np.asarray(algo_result["min_vector"])
    eigvecs[i] = algo_result["min_vector"]
print('\rOptimization complete      ');

In [ ]:
#%%capture out
fig, axs = plt.subplots(2,2, figsize=(15, 10), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .5, wspace=.001)
axs = axs.ravel()

for i in range(len(optimizers)):
    
    pltobj = axs[i].bar(eigvecs[i].keys(), eigvecs[i].values(), color='lightblue')
    pltobj = axs[i].tick_params('x', labelrotation=45)
    pltobj = axs[i].set_xlabel('Eigenvectors')
    pltobj = axs[i].set_ylabel('Counts')
    pltobj = axs[i].set_title('Optimizer: {}'.format(optimizers[i].__name__))
  

In [ ]:
## save VQE results
#filename_vqe = args.outpath_vqe \
#             + args.molecule + '_' \
#             + args.basis_set + '_' \
#             + d_string + '_VQE_SPSA_RY_full_bk_' \
#             + str(args.num_shots) + '.pkl'
#filehandler_vqe = open(filename_vqe, 'wb')
#pickle.dump(result, filehandler_vqe)
#print("saved file = ", filename_vqe)

In [ ]:
#print("VQE settings = ", algo.print_settings())
print("VQE result = ", algo_result)

In [ ]:
## load VQE results
#print(filename_vqe)
#result_load = pickle.load(open(filename_vqe,'rb'))
#result_load

## H2 321G basis set

In [ ]:
## setup molecule and pass in basis set
args.basis_set='321g'
molecule = 'H .0 .0 -0.25; H .0 .0 0.25'
driver = PySCFDriver(molecule, basis=args.basis_set)
qmolecule = driver.run()
operator =  Hamiltonian(transformation=TransformationType.FULL,
                        qubit_mapping=QubitMappingType.PARITY,
                        two_qubit_reduction=False,
                        freeze_core=False,
                        orbital_reduction=None)
qubit_op, aux_ops = operator.run(qmolecule)

## print some info
num_particles = qmolecule.num_alpha + qmolecule.num_beta
print("num orbitals = ", qmolecule.num_orbitals)
print("num_particles = ", num_particles)
print("num qubits = ", qubit_op.num_qubits)

In [ ]:
converge_cnts_321g = np.empty([len(optimizers)], dtype=object)
converge_vals_321g = np.empty([len(optimizers)], dtype=object)
eigvecs_321g = np.empty([len(optimizers)], dtype=object)

for i in range(len(optimizers)):
    aqua_globals.random_seed = 250
    optimizer = optimizers[i]()
    print('\rOptimizer: {}        '.format(type(optimizer).__name__), end='')
    #var_form = RY(num_qubits, initial_state=init_state)
    var_form = RYRZ(qubit_op.num_qubits, depth=args.vqe_depth, entanglement=args.vqe_entangler) 

    counts = []
    values = []
    def store_intermediate_result(eval_count, parameters, mean, std):
        counts.append(eval_count)
        values.append(mean)
  
    algo = VQE(qubit_op, var_form, optimizer, callback=store_intermediate_result)
    backend = Aer.get_backend('qasm_simulator')
    quantum_instance = QuantumInstance(circuit_caching=True, backend=backend,
                                       backend_options={'max_parallel_threads': args.max_parallel_threads,                                                                             'max_parallel_experiments': 0, 
                                                        'shots': args.num_shots})

    algo_result_321g = algo.run(quantum_instance)
    converge_cnts_321g[i] = np.asarray(counts)
    converge_vals_321g[i] = np.asarray(values)
    eigvecs_321g[i] = np.asarray(algo_result["min_vector"])
print('\rOptimization complete      ');

In [ ]:
print("VQE result = ", algo_result_321g)

In [ ]:
pylab.rcParams['figure.figsize'] = (12, 8)
for i in range(len(optimizers)):
    pylab.plot(converge_cnts_321g[i], converge_vals_321g[i], label=optimizers[i].__name__)
pylab.xlabel('Eval count')
pylab.ylabel('Energy')
pylab.title('Energy convergence for various optimizers, H2 321G')
pylab.legend(loc='upper right')

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(15, 10), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .5, wspace=.001)
axs = axs.ravel()

for i in range(len(optimizers)):
    
    pltobj = axs[i].bar(eigvecs[i].keys(), eigvecs[i].values(), color='lightblue')
    pltobj = axs[i].tick_params('x', labelrotation=45)
    pltobj = axs[i].set_xlabel('Eigenvectors')
    pltobj = axs[i].set_ylabel('Counts')
    pltobj = axs[i].set_title('Optimizer: {}'.format(optimizers[i].__name__))

In [ ]:
fig, axs = plt.subplots(2,1, figsize=(12,8), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace=0.5, wspace=0.001)

num_basis_sets = 2
for i in range(num_basis_sets):
    pylab.plot(converge_cnts[i], converge_vals[i], label=optimizers[i].__name__)
    pylab.xlabel('Eval count')
    pylab.ylabel('Energy')
    pylab.title('Energy convergence for various optimizers, H2 STO3G')
    pylab.legend(loc='upper right')